## Quick Start:

1. Make sure your `.env` file has `OPENAI_API_KEY`
2. Install dependencies: `uv sync`
3. Select "Vika Bot (Python 3.13)" kernel in Jupyter
4. Run all cells
5. Change the question and experiment!

The template will show you:

- Response from OpenAI
- Token usage (prompt/completion/total)
- **Cost breakdown in USD** (prompt cost + completion cost + total)


In [1]:
# Import libraries
import os
from dotenv import load_dotenv
from tokencost import calculate_prompt_cost, calculate_completion_cost
from IPython.display import display, Markdown
from agents import Agent, Runner, SQLiteSession, ModelSettings

load_dotenv()

MODEL = os.getenv("DEFAULT_MODEL", "gpt-4o-mini")
MAX_TOKENS = int(os.getenv("MAX_TOKENS", 1000))
TEMPERATURE = float(os.getenv("TEMPERATURE", 0.7))

print(f"Model: {MODEL}")
print(f"Max tokens: {MAX_TOKENS}")
print(f"Temperature: {TEMPERATURE}")


Model: gpt-4o-mini
Max tokens: 1000
Temperature: 0.7


In [2]:
# Structured prompt inputs
from pathlib import Path

# Load CV from markdown to keep parity with COVER_LETTER.ipynb
CV_PATH = Path("notebooks/assets/CV.md")
CV = CV_PATH.read_text(encoding="utf-8") if CV_PATH.exists() else ""

VACANCY = """
Frontend-разработчик — React
#удаленка #офис #senior
Компания: Т-Банк
☑️Обязанности
-Разрабатывать клиентские и внутренние интерфейсы и сопутствующие инструменты
-Внедрять новые технологии и практики
-Обучаться, совершенствовать свои знания и навыки

☑️Требования
-У вас есть опыт коммерческой разработки на React от 4-лет
-Отлично знаете Javascript
-Понимаете современный стек веб-разработки
-Готовы работать в распределенной команде
-Будет плюсом знание Typescript и опыт работы с ним, навык написания unit-тестов, опыт работы с Node.js, навык работы с CI/CD — GitLab
Контакты: https://www.tbank.ru/career/it/front-end-razrabotka/frontend-developer-react/
"""

# Build agent instructions (system prompt) mirroring COVER_LETTER.ipynb
INSTRUCTIONS = """
Ты эксперт по составлению сопроводительных писем с 10+ летним опытом работы в HR и рекрутменте. 
Твоя задача — создавать персонализированные, убедительные и релевантные сопроводительные письма, 
которые максимально точно соответствуют требованиям конкретной вакансии и подчеркивают сильные стороны кандидата.

Работай по структуре и правилам:
- Заголовок: "Отклик на вакансию [Название должности] - [Имя кандидата]"
- Приветствие (1 предложение), краткое представление и позиция
- Основная часть (2 предложения): конкретные навыки из вакансии, примеры опыта, количественные результаты
- Заключение (1 предложение): заинтересованность, готовность к собеседованию, вежливое завершение
- Подпись: контакты

Стиль: формально, уверенно, конкретно, кратко (300–600 знаков).
Избегай фраз, начинающихся с "Я"; избегай клише и длинных предложений.
Используй ключевые слова вакансии, добавляй цифры и результаты.
Не переносить опыт Tekara и Хроника на другие компании.

Входные данные ниже в теге resume и vacancy. Верни результат строго в формате:
```
**Тема:**
[Тема письма]

**Текст письма:**
[Полный текст сопроводительного письма]
```
"""

# Compose the user input that the agent will receive
USER_INPUT = f"""
<resume>
{CV}
</resume>

<vacancy>
{VACANCY}
</vacancy>
"""

print("Inputs prepared. Ready to run the agent.")

Inputs prepared. Ready to run the agent.


In [3]:
# Build a cover letter agent using OpenAI Agents SDK
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions

cover_letter_agent = Agent(
    name="Cover Letter Agent",
    instructions=prompt_with_handoff_instructions(INSTRUCTIONS),
    model=MODEL,
    model_settings=ModelSettings(
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
    ),
)

# Optional persistent memory between runs
session = SQLiteSession("cover_letter_session")

# Run the agent
try:
    result = await Runner.run(
        starting_agent=cover_letter_agent,
        input=USER_INPUT,
        session=session,
    )

    answer = result.final_output
    display(Markdown(answer))

    # Naive cost estimation using tokencost, aligned with your other notebook conventions
    try:
        messages = [{"role": "user", "content": USER_INPUT}]
        prompt_cost = calculate_prompt_cost(messages, MODEL)
        completion_cost = calculate_completion_cost(answer, MODEL)
        print(f"Total cost (USD): ${prompt_cost + completion_cost:.6f}")
    except Exception as cost_error:
        print(f"Could not calculate cost: {cost_error}")

except Exception as e:
    print(f"❌ Error: {e}")
    print("Make sure your OPENAI_API_KEY is set in .env file")


```
**Тема:**
Отклик на вакансию Frontend-разработчик — React - [Имя кандидата]

**Текст письма:**
Уважаемые представители Т-Банк, обращаюсь по поводу вакансии Frontend-разработчика. Обладаю более 5 лет опыта в коммерческой разработке на React и отличными знаниями Javascript, что позволяет создавать высококачественные клиентские интерфейсы.

В своей предыдущей работе внедрил новые технологии, что увеличило производительность команды на 30%. Также имею опыт работы с Typescript и CI/CD — GitLab, что дополнительно улучшает качество разработки. Готов к обучению и совершенствованию навыков в распределенной команде.

С нетерпением жду возможности обсудить свою кандидатуру на собеседовании. Спасибо за внимание.

С уважением,  
[Имя кандидата]  
[Телефон]  
[Email]
```

Total cost (USD): $0.000149
